In [1]:
import os
import openai

from decouple import Config, RepositoryEnv
config = Config(RepositoryEnv("/workspaces/codespaces-jupyter/.env"))

openai.api_key = config("OPENAI_API_KEY")

In [2]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions}" \
    + f" questions. Each question should have {num_possible_answers} options. " \
    + f"Also include the correct answer for each question using the starting string 'Correct Answer:'. " \

    return prompt

In [3]:
create_test_prompt("Geography", 4, 4)

"Create a multiple choice quiz on the topic of Geography consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer:'. "

In [4]:
response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=create_test_prompt('vietnam\'s History', 4, 4),
    max_tokens=256,
    temperature=0.7
)

In [5]:
print(response['choices'][0]['text'])



Q1. When did the Vietnam War officially end?
A. 1964
B. 1975
C. 1985
D. 1995
Correct Answer: B. 1975

Q2. Who was the first president of North Vietnam?
A. Ho Chi Minh
B. Ngo Dinh Diem
C. Nguyen Van Thieu
D. Tran Trong Kim
Correct Answer: A. Ho Chi Minh

Q3. What religion is popular in the nation of Vietnam?
A. Buddhism
B. Christianity
C. Judaism
D. Hinduism
Correct Answer: A. Buddhism

Q4. What country occupied Vietnam during World War II?
A. China
B. Japan
C. France
D. Germany
Correct Answer: B. Japan


In [6]:
def create_student_view(quiz_content, num_questions):
    student_view = {1: ''}
    ques_number = 1
    for line in quiz_content.split('\n'):
        if not line.startswith("Correct Answer:"):
            student_view[ques_number] += line + "\n"
        else:
            if ques_number < num_questions:
                ques_number += 1
                student_view[ques_number] = ''

    return student_view

In [7]:
std_view = create_student_view(response['choices'][0]['text'], 4)

In [8]:
for key in std_view:
    print(std_view[key])



Q1. When did the Vietnam War officially end?
A. 1964
B. 1975
C. 1985
D. 1995


Q2. Who was the first president of North Vietnam?
A. Ho Chi Minh
B. Ngo Dinh Diem
C. Nguyen Van Thieu
D. Tran Trong Kim


Q3. What religion is popular in the nation of Vietnam?
A. Buddhism
B. Christianity
C. Judaism
D. Hinduism


Q4. What country occupied Vietnam during World War II?
A. China
B. Japan
C. France
D. Germany



In [9]:
def extract_answer(quiz_content, num_questions):
    answers = {1: ''}
    ques_number = 1
    for line in quiz_content.split('\n'):
        if line.startswith("Correct Answer:"):
            answers[ques_number] += line + "\n"

            if ques_number < num_questions:
                ques_number += 1
                answers[ques_number] = ''

    return answers

In [16]:
correct_ans = extract_answer(response['choices'][0]['text'], 4)
print(correct_ans)

{1: 'Correct Answer: B. 1975\n', 2: 'Correct Answer: A. Ho Chi Minh\n', 3: 'Correct Answer: A. Buddhism\n', 4: 'Correct Answer: B. Japan\n'}


In [12]:
def take(student_view):
    student_ans = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_ans[question] = answer

    return student_ans

In [13]:
student_answers = take(student_view=std_view)



Q1. When did the Vietnam War officially end?
A. 1964
B. 1975
C. 1985
D. 1995


Q2. Who was the first president of North Vietnam?
A. Ho Chi Minh
B. Ngo Dinh Diem
C. Nguyen Van Thieu
D. Tran Trong Kim


Q3. What religion is popular in the nation of Vietnam?
A. Buddhism
B. Christianity
C. Judaism
D. Hinduism


Q4. What country occupied Vietnam during World War II?
A. China
B. Japan
C. France
D. Germany



In [14]:
print(student_answers)

{1: 'B', 2: 'A', 3: 'A', 4: 'B'}


In [19]:
correct_ans[1][16]

'B'

In [20]:
def grade(correct_ans, student_ans):
    score = 0
    for ques, ans in student_ans.items():
        if ans.upper() == correct_ans[ques][16]:
            score += 1
        
    grade = 100 * score / len(correct_ans)

    if grade < 60:
        passed = "NO PASS!"
    else:
        passed = "PASS!"

    return f"{score}/{len(correct_ans)} correct! You got {grade} grade, {passed}"

In [21]:
grade(correct_ans, student_answers)

'4/4 correct! You got 100.0 grade, PASS!'